In [3]:
# https://platform.openai.com/docs/guides/function-calling


In [6]:
from semantic_router.schema import Route

from semantic_router.encoders import CohereEncoder
from semantic_router.layer import RouteLayer

encoder = CohereEncoder()

config = [
    {
        "name": "get_weather",
        "utterances": [
            "What is the weather like in SF?",
            "What is the weather in Cyprus?",
            "weather in London?",
        ],
    },
    {
        "name": "get_time",
        "utterances": [
            "What time is it in New York?",
            "What time is it in London?",
            "What is the time in Paris?",
        ],
    },
    {
        "name": "get_news",
        "utterances": [
            "What is happening in the world?",
            "What is the latest news?",
            "What is the latest news in the US?",
        ],
    },
]

routes = [Route(name=route["name"], utterances=route["utterances"]) for route in config]

route_layer = RouteLayer(encoder=encoder, routes=routes)

queries = [
    "What is the weather like in Barcelona?",
    "What time is it in Taiwan?",
    "What is happening in the world?",
]

for query in queries:
    function_name = route_layer(query)
    print(function_name)

get_weather
get_time
get_news


In [ ]:
def get_weather(location: str):
    print(f"getting weather for {location}")


def extract_function_parameters(query: str, function: Callable):
    # llm(
    #     query=query,
    #     function=function,
    #     prompt="What are the parameters for this function?",
    # )
    print("Extracting function parameters..")


if category == "get_weather":
    print(f"Category is `{category}`")
    params = extract_function_parameters(query, get_weather)
    print("Getting weather..")
    # get_weather(**params)

In [10]:
print(generated_config)

None


In [21]:
import json

example_specification = (
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        },
    },
)

example_config = {
    "name": "get_weather",
    "utterances": [
        "What is the weather like in SF?",
        "What is the weather in Cyprus?",
        "weather in London?",
    ],
}

specification = (
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get the current time",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location"],
            },
        },
    },
)

prompt = f"""
    Given the following specification, generate a config in JSON format
    Example:
    SPECIFICATION:
    {example_specification}

    CONFIG:
    {example_config}

    GIVEN SPECIFICATION:
    {specification}

    GENERATED CONFIG:
"""


response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": f"{prompt}"},
    ],
)

ai_message = response.choices[0].message.content
if ai_message:
    route_config = json.loads(ai_message)
    print(f"Route config: {route_config}")


Route config: {'name': 'get_time', 'utterances': ['What is the current time in San Francisco?', 'What time is it in New York?', 'Current time in London?']}


In [23]:
routes = [Route(name=route["name"], utterances=route["utterances"]) for route in [route_config]]

route_layer = RouteLayer(encoder=encoder, routes=routes)

queries = [
    "What is the weather like in Barcelona?",
    "What time is it in Taiwan?",
    "What is happening in the world?",
    "what is the time in Kaunas?"
    "Im bored",
    "I want to play a game",
    "Banana"
]

for query in queries:
    function_name = route_layer(query)
    print(function_name)

None
get_time
get_time
get_time
None
None
